In [33]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [34]:
hospitals = pd.read_csv('../data/non-residential_facilities/hospitals.csv')
schools = pd.read_csv('../data/non-residential_facilities/schools.csv')
shopping = pd.read_csv('../data/non-residential_facilities/shopping_centre.csv')
sport = pd.read_csv('../data/non-residential_facilities/sport_facilities.csv')

In [35]:
def to_gpd(df):
    df['geometry'] = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs('EPSG:4326', inplace=True)
    gdf = gdf.to_crs('EPSG:7855')
    return gdf

hospitals = to_gpd(hospitals)
schools = to_gpd(schools)
shopping = to_gpd(shopping)
sport = to_gpd(sport)

In [36]:
GCCSA = gpd.read_file('../data/GCCSA_2021_AUST_SHP_GDA2020/GCCSA_2021_AUST_GDA2020.shp')
GCCSA = GCCSA.to_crs('EPSG:7855')
Greater_Melbourne = GCCSA.iloc[4]['geometry']

SA2 = gpd.read_file('../data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
SA2 = SA2.to_crs('EPSG:7855')
SA2_GreaterMelbourne = SA2[SA2.within(Greater_Melbourne)]
SA2_GreaterMelbourne = SA2_GreaterMelbourne[['SA2_CODE21', 'SA2_NAME21', 'geometry']]
SA2_GreaterMelbourne['SA2_CODE21'] = SA2_GreaterMelbourne['SA2_CODE21'].astype('int64')

In [37]:
joined_hospital = gpd.sjoin(hospitals, SA2_GreaterMelbourne, how="inner", predicate='within')
joined_schools = gpd.sjoin(schools, SA2_GreaterMelbourne, how="inner", predicate='within')
joined_shopping = gpd.sjoin(shopping, SA2_GreaterMelbourne, how="inner", predicate='within')
joined_sport = gpd.sjoin(sport, SA2_GreaterMelbourne, how="inner", predicate='within')

In [38]:
joined_hospital = joined_hospital.drop(['Postcode', 'Latitude', 'Longitude', 'index_right'], axis=1)
joined_hospital.to_csv('../output/preprocessing/non-residential_facilities/hospitals_GreaterMelbourne.csv', index=False)

In [39]:
joined_schools = joined_schools.drop(['Address_Postcode', 'Latitude', 'Longitude', 'index_right'], axis=1)
joined_schools.to_csv('../output/preprocessing/non-residential_facilities/schools_GreaterMelbourne.csv', index=False)

In [40]:
joined_shopping = joined_shopping.drop(['Latitude', 'Longitude', 'index_right'], axis=1)
joined_shopping.to_csv('../output/preprocessing/non-residential_facilities/shopping_GreaterMelbourne.csv', index=False)

In [41]:
joined_sport = joined_sport.drop(['Pcode', 'Latitude', 'Longitude', 'index_right'], axis=1)
joined_sport.to_csv('../output/preprocessing/non-residential_facilities/sport_GreaterMelbourne.csv', index=False)